## Download datasets

<!--
import data_analytics.github as github
print(github.create_jupyter_notebook_header("tahirawwad", "agriculture-data-analytics", "notebooks/notebook-1-01-dc-cso-api.ipynb", "master"))
-->
<table style="margin: auto;"><tr><td><a href="https://mybinder.org/v2/gh/tahirawwad/agriculture-data-analytics/master?filepath=notebooks/notebook-1-01-dc-cso-api.ipynb" target="_parent"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a></td><td>online editors</td><td><a href="https://colab.research.google.com/github/tahirawwad/agriculture-data-analytics/blob/master/notebooks/notebook-1-01-dc-cso-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></td></tr></table>

### Objective

The objective is to download the datasets with the listed in [asset-link-builder.xlsx](./../artifacts/asset-link-builder.xlsx) from [cso.ie](https://wwww.cso.ie).  

### Setup

Import required third party Python libraries, import supporting functions and sets up data source file paths.

In [1]:
# Local
#!pip install -r script/requirements.txt
# Remote option
#!pip install -r https://raw.githubusercontent.com/tahirawwad/agriculture-data-analytics/requirements.txt
#Options: --quiet --user

In [2]:
from pandas import DataFrame
import io
import pandas
import requests

In [3]:
def download_cso_table_data(table_code: str,
                            file_type: str = "CSV",
                            version: str = "1.0") -> str:
    BASE_URL = "https://ws.cso.ie/public/api.jsonrpc"
    JSON_DATA = f'{{"jsonrpc":"2.0","method":"PxStat.Data.Cube_API.ReadDataset","params":{{"class":"query","id":[],"dimension":{{}},"extension":{{"pivot":null,"codes":false,"language":{{"code":"en"}},"format":{{"type":"{file_type}","version":"{version}"}},"matrix":"{table_code}"}},"version":"2.0"}}}}'

    url = f"{BASE_URL}?data={JSON_DATA}"
    response_json_rpc = requests.get(url).json()
    print(f"Downloaded https://data.cso.ie/table/{table_code}")
    return response_json_rpc['result']


def download_cso_table_dataframe(table_code: str) -> str:
    return pandas.read_csv(io.StringIO(download_cso_table_data(table_code)),
                           sep=",")

### Download CSO Data sources

In [4]:
filepath = './../artifacts/asset-link-builder.xlsx'

excelWorkbook = pandas.ExcelFile(filepath)

cso_datasources_dataframe: DataFrame = excelWorkbook.parse('CSO Tables')
excelWorkbook.close()

In [5]:
cso_datasources_dataframe = cso_datasources_dataframe[cso_datasources_dataframe['Download Date'] == '2022-01-19']
cso_datasources_dataframe[['Code', 'Title']]

,Code,Title
2,AEA01,"Value at Current Prices for Output, Input and ..."
3,AEA05,Value at Current Prices for Subsidies on Products
4,AHA01,Agricultural Input and Output Price Indices
5,AHA03,Agricultural Input and Output Price Indices
6,AHA04,Agricultural Input and Output Price Indices
10,AQA03,Crop Yield 1985-2007
11,AQA04,Crop Yield and Production
16,DAA01,Bovine Tuberculosis
19,TSA04,Exports of Cattle and Beef 1930-2020


In [6]:
for _, cso_datasource in cso_datasources_dataframe.iterrows():
    print("Get", cso_datasource['Code'], cso_datasource['Title'])
    dataframe = download_cso_table_dataframe(cso_datasource['Code'])
    filepath: str = f"./../assets/{cso_datasource['Filename']}"
    dataframe.to_csv(filepath)
    print(filepath)

Get AEA01 Value at Current Prices for Output, Input and Income in Agriculture
Downloaded https://data.cso.ie/table/AEA01
./../assets/cso-aea01-value-at-current-prices-for-output-input-and-income-in-agriculture.csv
Get AEA05 Value at Current Prices for Subsidies on Products
Downloaded https://data.cso.ie/table/AEA05
./../assets/cso-aea05-value-at-current-prices-for-subsidies-on-products.csv
Get AHA01 Agricultural Input and Output Price Indices
Downloaded https://data.cso.ie/table/AHA01
./../assets/cso-aha01-agricultural-input-and-output-price-indices.csv
Get AHA03 Agricultural Input and Output Price Indices
Downloaded https://data.cso.ie/table/AHA03
./../assets/cso-aha03-agricultural-input-and-output-price-indices.csv
Get AHA04 Agricultural Input and Output Price Indices
Downloaded https://data.cso.ie/table/AHA04
./../assets/cso-aha04-agricultural-input-and-output-price-indices.csv
Get AQA03 Crop Yield 1985-2007
Downloaded https://data.cso.ie/table/AQA03
./../assets/cso-aqa03-crop-yield

### Normalize CSO Data sources

#### AEA01 Value at Current Prices for Output, Input and Income in Agriculture

In [7]:
filepath: str = './../assets/cso-aea01-value-at-current-prices-for-output-input-and-income-in-agriculture.csv'
prc_9021df = pandas.read_csv(filepath).pivot_table(columns="Statistic",
                                                   index=['Year', 'UNIT'],
                                                   values='VALUE',
                                                   dropna=True).reset_index()

prc_9021df.to_csv('./../artifacts/TA_inputoutputvalue_1990_2021_CSO.csv')

#### AEA05 Value at Current Prices for Subsidies on Products

In [8]:
filepath: str = './../assets/cso-aea05-value-at-current-prices-for-subsidies-on-products.csv'

subsidies_df = pandas.read_csv(filepath).pivot_table(
    columns="Statistic", index=['Year', 'UNIT'], values='VALUE',
    dropna=True).reset_index()

subsidies_df.to_csv('./../artifacts/TA_subsidies_1990_2020_CSO.csv')

#### AHA01 Agricultural Input and Output Price Indices

In [9]:
filepath: str = './../assets/cso-aha01-agricultural-input-and-output-price-indices.csv'

prc_idx_9510_df = pandas.read_csv(filepath).pivot_table(
    columns="Agricultural Product",
    index=['Year', 'UNIT'],
    values='VALUE',
    dropna=True).reset_index()

prc_idx_9510_df.to_csv(
    './../artifacts/TA_inputoutputpriceIDX_1995_2010_CSO.csv')

#### AHA03 Agricultural Input and Output Price Indices

In [10]:
filepath: str = './../assets/cso-aha03-agricultural-input-and-output-price-indices.csv'

prc_idx_0517_df = pandas.read_csv(filepath).pivot_table(
    columns="Agricultural Product",
    index=['Year', 'UNIT'],
    values='VALUE',
    dropna=True).reset_index()

prc_idx_0517_df.to_csv(
    './../artifacts/TA_inputoutputpriceIDX_2005_2017_CSO.csv')

#### AHA04 Agricultural Input and Output Price Indices (Base 2015=100)

In [11]:
filepath: str = './../assets/cso-aha04-agricultural-input-and-output-price-indices.csv'

prc_idx_1420_df = pandas.read_csv(filepath).pivot_table(
    columns="Agricultural Product",
    index=['Year', 'UNIT'],
    values='VALUE',
    dropna=True).reset_index()

prc_idx_1420_df.to_csv(
    './../artifacts/TA_inputoutputpriceIDX_2014_2020_CSO.csv')

#### AQA03 Crop Yield 1985-2007

In [12]:
filepath: str = './../assets/cso-aqa03-crop-yield-1985-2007.csv'

crop_yield8507_df = pandas.read_csv(filepath).pivot_table(
    columns="Statistic",
    index=['Year', 'Type of Crop', 'UNIT'],
    values='VALUE',
    dropna=True).reset_index()

crop_yield8507_df.to_csv('./../artifacts/TA_cropyield_1985_2007_CSO.csv')

#### AQA04 Crop Yield and Production

In [13]:
filepath: str = './../assets/cso-aqa04-crop-yield-and-production.csv'

crop_yield0820_df = pandas.read_csv(filepath).pivot_table(
    columns="Statistic",
    index=['Year', 'Type of Crop', 'UNIT'],
    values='VALUE',
    dropna=True).reset_index().rename(
        columns={"Crop Production": "Crop Yield"})

crop_yield0820_df.to_csv('./../artifacts/TA_cropyield_2008_2020_CSO.csv')

Join Crop Yields from 1985 to 2020 into 1 dataframe

In [14]:
# append crop yield from 1985 tp 2020
crop_yield_ie_df = crop_yield8507_df.append(crop_yield0820_df)

crop_yield_ie_df.to_csv('./../artifacts/TA_cropyield_1985_2020_CSO.csv')